<a href="https://colab.research.google.com/github/rerebebeccacca/recommend_system20210713/blob/main/My_Content_Based_Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content-Based Movie Recommendation
https://www.kaggle.com/annalee7/content-based-movie-recommendation-engine

## 安裝自然語言處理的套件

In [ ]:
! pip install rake_nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7829 sha256=11a068b76b7871dbf1f8752e090f25c4840de928147f731f64332ab7822e99c2
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [ ]:
import pandas as pd
import numpy as np
from rake_nltk import Rake

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from ast import literal_eval

## 合併兩個檔案

In [ ]:
movies = pd.read_csv('./tmdb_5000_movies.csv')
credits = pd.read_csv('./tmdb_5000_credits.csv')

# Join datasets
credits.columns = ['id', 'title', 'cast', 'crew']

alldata = movies.merge(credits, on = 'id')
alldata.head()

ParserError: ignored

## Json格式欄位處理函數

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Trim dataset to include relevant features
df = alldata[['id', 'original_title', 'genres', 'keywords', 'overview', 'original_language', 'cast', 'crew']]

# Parse stringed list features into python objects
features = ['keywords', 'genres', 'cast', 'crew']
for i in features:
    df[i] = alldata[i].apply(literal_eval)
    
# Extract list of genres
def list_genres(x):
    l = [d['name'] for d in x]
    return(l)
df['genres'] = df['genres'].apply(list_genres)

# Extract top 3 cast members
def list_cast(x):
    l = [d['name'] for d in x]
    if len(l) > 3:
        l = l[:3]
    return(l)
df['cast'] = df['cast'].apply(list_cast)

# Extract top 5 keywords
def list_keywords(x):
    l = [d['name'] for d in x]
    if len(l) > 5:
        l = l[:5]
    return(l)
df['keywords'] = df['keywords'].apply(list_keywords)

# Extract director
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
df['director'] = df['crew'].apply(get_director)

# Drop the now unnecessary crew feature
df = df.drop('crew', axis = 1)

# Clean features of spaces and lowercase all to ensure uniques
def clean_feat(x):
    if isinstance(x, list):
        return [i.lower().replace(" ","") for i in x]
    else:
        if isinstance(x, str):
            return x.lower().replace(" ", "")
        else:
            return ''

## Json格式欄位處理，找出關鍵字、風格、演員、導演

In [ ]:
features = ['keywords', 'genres', 'cast', 'director']
for i in features:
    df[i] = df[i].apply(clean_feat)
df.head()

,id,original_title,genres,keywords,overview,original_language,cast,director
0,19995,Avatar,"[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...","In the 22nd century, a paraplegic Marine is di...",en,"[samworthington, zoesaldana, sigourneyweaver]",jamescameron
1,285,Pirates of the Caribbean: At World's End,"[adventure, fantasy, action]","[ocean, drugabuse, exoticisland, eastindiatrad...","Captain Barbossa, long believed to be dead, ha...",en,"[johnnydepp, orlandobloom, keiraknightley]",goreverbinski
2,206647,Spectre,"[action, adventure, crime]","[spy, basedonnovel, secretagent, sequel, mi6]",A cryptic message from Bond’s past sends him o...,en,"[danielcraig, christophwaltz, léaseydoux]",sammendes
3,49026,The Dark Knight Rises,"[action, crime, drama, thriller]","[dccomics, crimefighter, terrorist, secretiden...",Following the death of District Attorney Harve...,en,"[christianbale, michaelcaine, garyoldman]",christophernolan
4,49529,John Carter,"[action, adventure, sciencefiction]","[basedonnovel, mars, medallion, spacetravel, p...","John Carter is a war-weary, former military ca...",en,"[taylorkitsch, lynncollins, samanthamorton]",andrewstanton


## 去除遺漏值(Missing Value)

In [ ]:
missing = df.columns[df.isnull().any()]
df[missing].isnull().sum().to_frame()
# Replace NaN from overview with an empty string
df['overview'] = df['overview'].fillna('')
df.head()

,id,original_title,genres,keywords,overview,original_language,cast,director
0,19995,Avatar,"[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...","In the 22nd century, a paraplegic Marine is di...",en,"[samworthington, zoesaldana, sigourneyweaver]",jamescameron
1,285,Pirates of the Caribbean: At World's End,"[adventure, fantasy, action]","[ocean, drugabuse, exoticisland, eastindiatrad...","Captain Barbossa, long believed to be dead, ha...",en,"[johnnydepp, orlandobloom, keiraknightley]",goreverbinski
2,206647,Spectre,"[action, adventure, crime]","[spy, basedonnovel, secretagent, sequel, mi6]",A cryptic message from Bond’s past sends him o...,en,"[danielcraig, christophwaltz, léaseydoux]",sammendes
3,49026,The Dark Knight Rises,"[action, crime, drama, thriller]","[dccomics, crimefighter, terrorist, secretiden...",Following the death of District Attorney Harve...,en,"[christianbale, michaelcaine, garyoldman]",christophernolan
4,49529,John Carter,"[action, adventure, sciencefiction]","[basedonnovel, mars, medallion, spacetravel, p...","John Carter is a war-weary, former military ca...",en,"[taylorkitsch, lynncollins, samanthamorton]",andrewstanton


## BOW(bag of keywords)

## 找出關鍵字

In [ ]:
# Initialize empty column
df['plotwords'] = ''

# function to get keywords from a text
def get_keywords(x):
    plot = x
    
    # initialize Rake using english stopwords from NLTK, and all punctuation characters
    rake = Rake()
    
    # extract keywords from text
    # https://pypi.org/project/rake-nltk/
    rake.extract_keywords_from_text(plot)
    
    # get dictionary with keywords and scores
    scores = rake.get_word_degrees()
    
    # return new keywords as list, ignoring scores
    return(list(scores.keys()))

# Apply function to generate keywords
df['plotwords'] = df['overview'].apply(get_keywords)

## 自關鍵字、風格、演員、導演欄位找出重要的單字

In [ ]:
df_keys = pd.DataFrame() 

df_keys['title'] = df['original_title']
df_keys['keywords'] = ''

def bag_words(x):
    return(' '.join(x['genres']) + ' ' + ' '.join(x['keywords']) + ' ' +  ' '.join(x['cast']) + 
           ' ' + ' '.join(x['director']) + ' ' + ' '.join(x['plotwords']))
df_keys['keywords'] = df.apply(bag_words, axis = 1)

df_keys.head()

,title,keywords
0,Avatar,action adventure fantasy sciencefiction cultur...
1,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,Spectre,action adventure crime spy basedonnovel secret...
3,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,John Carter,action adventure sciencefiction basedonnovel m...


## 使用 BOW，將每一部電影的文字內容，轉成詞向量

In [ ]:
# 將文件中的詞語轉換為詞頻矩陣
cv = CountVectorizer()
# 計算個詞語出現的次數
cv_mx = cv.fit_transform(df_keys['keywords'])

## 計算相似性(cosine similarity)

In [ ]:
# create cosine similarity matrix
cosine_sim = cosine_similarity(cv_mx, cv_mx)
cosine_sim

array([[1.        , 0.10527936, 0.06299408, ..., 0.        , 0.        ,
        0.        ],
       [0.10527936, 1.        , 0.09284767, ..., 0.02438299, 0.        ,
        0.        ],
       [0.06299408, 0.09284767, 1.        , ..., 0.02188441, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02438299, 0.02188441, ..., 1.        , 0.05938557,
        0.03959038],
       [0.        , 0.        , 0.        , ..., 0.05938557, 1.        ,
        0.06818182],
       [0.        , 0.        , 0.        , ..., 0.03959038, 0.06818182,
        1.        ]])

## 建立電影名稱與列索引對照表，以利查詢

In [ ]:
# create list of indices for later matching
indices = pd.Series(df_keys.index, index = df_keys['title'])

## 推薦(Recommendation) 處理函數，輸入參數為電影名稱，輸出為前N名相似的電影

In [ ]:
def recommend_movie(title, n = 10, cosine_sim = cosine_sim):
    movies = []
    
    # retrieve matching movie title index
    if title not in indices.index:
        print("Movie not in database.")
        return
    else:
        idx = indices[title]
    
    # cosine similarity scores of movies in descending order
    scores = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    
    # top n most similar movies indexes
    # use 1:n because 0 is the same movie entered
    top_n_idx = list(scores.iloc[1:n].index)
        
    return df_keys['title'].iloc[top_n_idx]

## 測試：找出Toy Story這部電影的前5部最相似的電影

In [ ]:
recommend_movie('Toy Story', n = 5)

343        Toy Story 2
42         Toy Story 3
221    Stuart Little 2
837         Free Birds
Name: title, dtype: object

## 測試：找出The Avengers這部電影的前10部最相似的電影

In [ ]:
recommend_movie('The Avengers')

7                  Avengers: Age of Ultron
174                    The Incredible Hulk
511                                  X-Men
85     Captain America: The Winter Soldier
79                              Iron Man 2
26              Captain America: Civil War
169     Captain America: The First Avenger
31                              Iron Man 3
68                                Iron Man
Name: title, dtype: object